In [11]:
from datetime import timedelta
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Full table preprocessed

In [12]:
fds = pd.read_csv('/Users/polotailokyin/Desktop/GE1356/Project/data_collected/corona-virus-report/covid_19_clean_complete.csv',
                         parse_dates=['Date'])
ship_rows = fds['Province/State'].str.contains('Grand Princess') | fds['Province/State'].str.contains('Diamond Princess') | fds['Country/Region'].str.contains('Diamond Princess') | fds['Country/Region'].str.contains('MS Zaandam')
ship = fds[ship_rows]
fds = fds[~(ship_rows)]
fds['Active'] = fds['Confirmed'] - fds['Deaths'] - fds['Recovered']
fds[['Province/State']] = fds[['Province/State']].fillna('')
fds[['Confirmed', 'Deaths', 'Recovered', 'Active']] = fds[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)
fds['Recovered'] = fds['Recovered'].astype(int)

fds.to_csv('full_preprocessed.csv', index=False)

# Date and country

In [14]:
fgdc = fds.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
t = fgdc.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
t = t.sum().diff().reset_index()
m = t['Country/Region'] != t['Country/Region'].shift(1)
t.loc[m, 'Confirmed'] = np.nan
t.loc[m, 'Deaths'] = np.nan
t.loc[m, 'Recovered'] = np.nan
t.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
fgdc = pd.merge(fgdc, t, on=['Country/Region', 'Date'])
fgdc = fgdc.fillna(0)
c = ['New cases', 'New deaths', 'New recovered']
fgdc[c] = fgdc[c].astype('int')
fgdc['New cases'] = fgdc['New cases'].apply(lambda x: 0 if x<0 else x)

fgdc["Date"]
fgdc.to_csv('date.csv')

# Day

In [4]:
d = fgdc.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()
d['Deaths / 100 Cases'] = round((d['Deaths']/d['Confirmed'])*100, 2)
d['Recovered / 100 Cases'] = round((d['Recovered']/d['Confirmed'])*100, 2)
d['Deaths / 100 Recovered'] = round((d['Deaths']/d['Recovered'])*100, 2)
d['No. of countries'] = fgdc[fgdc['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len).values
cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
d[cols] = d[cols].fillna(0)

d.to_csv('day.csv')

# Country

In [5]:
cw = fgdc[fgdc['Date']==max(fgdc['Date'])].reset_index(drop=True).drop('Date', axis=1)
cw = cw.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()
cw['Deaths / 100 Cases'] = round((cw['Deaths']/cw['Confirmed'])*100, 2)
cw['Recovered / 100 Cases'] = round((cw['Recovered']/cw['Confirmed'])*100, 2)
cw['Deaths / 100 Recovered'] = round((cw['Deaths']/cw['Recovered'])*100, 2)
cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
cw[cols] = cw[cols].fillna(0)
pop = pd.read_csv("/Users/polotailokyin/Desktop/GE1356/Project/data_collected/population-by-country/population_by_country_2020.csv")
pop = pop.iloc[:, :2]
pop.columns = ['Country/Region', 'Population']
cw = pd.merge(cw, pop, on='Country/Region', how='left')
cols = ['Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Czechia',
        'Kosovo', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines',
        'Taiwan*', 'US', 'West Bank and Gaza']
pops = [54409800, 89561403, 5518087, 26378274, 10708981, 1793000,
        53109, 110854, 23806638, 330541757, 4543126]
for c, p in zip(cols, pops):
    cw.loc[cw['Country/Region']== c, 'Population'] = p
cw['Cases / Million People'] = round((cw['Confirmed'] / cw['Population']) * 1000000)
today = fgdc[fgdc['Date']==max(fgdc['Date'])].reset_index(drop=True).drop('Date', axis=1)[['Country/Region', 'Confirmed']]
last_week = fgdc[fgdc['Date']==max(fgdc['Date'])-timedelta(days=7)].reset_index(drop=True).drop('Date', axis=1)[['Country/Region', 'Confirmed']]
t = pd.merge(today, last_week, on='Country/Region', suffixes=(' today', ' last week'))
t['1 week change'] = t['Confirmed today'] - t['Confirmed last week']
t = t[['Country/Region', 'Confirmed last week', '1 week change']]
cw = pd.merge(cw, t, on='Country/Region')
cw['1 week % increase'] = round(cw['1 week change']/cw['Confirmed last week']*100, 2)

cw.to_csv('country.csv')